In [1]:
import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
from gensim.models.poincare import PoincareModel
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from gensim.models.poincare import PoincareModel
from sklearn.model_selection import train_test_split
import random
import random
import numpy as np
import pandas as pd
from collections import defaultdict

from nltk.corpus import wordnet as wn
from gensim.models.poincare import PoincareModel
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoel1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Mammals Subset

In [4]:
nltk.download('wordnet')
def prepare_mammal_data(subset_size=10000, test_size=0.2, seed=42):
    def extract_transitive_hypernyms_mammals():
        mammal_synset = wn.synset('mammal.n.01')
        mammal_synsets = set(mammal_synset.closure(lambda s: s.hyponyms()))
        mammal_synsets.add(mammal_synset)

        relations = []
        for synset in mammal_synsets:
            paths = synset.hypernym_paths()
            for path in paths:
                for ancestor in path:
                    if ancestor in mammal_synsets and ancestor != synset:
                        relations.append((synset.name(), ancestor.name()))
        return relations

    print("Building transitive closure for the Mammal subtree...")
    relations_list = extract_transitive_hypernyms_mammals()
    print(f"Total transitive hypernym pairs (mammal subtree): {len(relations_list)}")

    
    if subset_size is not None and len(relations_list) > subset_size:
        random.seed(seed)
        relations_list = random.sample(relations_list, subset_size)
        print(f"Using a SUBSET of {subset_size} edges for faster testing.")

    
    train_relations, test_relations = train_test_split(
        relations_list, test_size=test_size, random_state=seed
    )

    print(f"Train relations: {len(train_relations)}")
    print(f"Test relations : {len(test_relations)}")

    combined_relations = train_relations + test_relations
    return train_relations, test_relations, combined_relations



def flip_edge_directions(relations, flip_probability=0.1, seed=42):
    random.seed(seed)
    flipped_relations = []
    for (child, ancestor) in relations:
        if random.random() < flip_probability:
            flipped_relations.append((ancestor, child))  
        else:
            flipped_relations.append((child, ancestor))  
    return flipped_relations



def train_and_evaluate_poincare(
    train_relations,
    test_relations,
    combined_relations,
    embedding_dim=5,
    epochs=300,
    n_negatives_strict=500,
):

   
    model = PoincareModel(
        train_data=train_relations,
        size=embedding_dim,
        negative=10,  
        burn_in=10    
    )
    model.train(epochs=epochs)

    
    u_to_all_neighbors = defaultdict(set)
    for (child, ancestor) in combined_relations:
        u_to_all_neighbors[child].add(ancestor)

    vocab_nodes = set(model.kv.index_to_key)
    vocab_list = list(vocab_nodes)
    all_edges_set = set(combined_relations)

    def reconstruction_mean_rank_strict_sampled(model, edges, n_negatives=500, seed=1234):
        rng = np.random.default_rng(seed)
        ranks = []
        for (u, v) in edges:
            if (u not in model.kv) or (v not in model.kv):
                continue
            neighbors = u_to_all_neighbors[u]
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < n_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate not in (u, v) and (candidate not in neighbors):
                    neg_candidates.append(candidate)
                attempts += 1
            candidates = neg_candidates + [v]
            dists = [(c, model.kv.distance(u, c)) for c in candidates]
            sorted_nodes = [x[0] for x in sorted(dists, key=lambda x: x[1])]
            try:
                rank = sorted_nodes.index(v) + 1
                ranks.append(rank)
            except ValueError:
                pass
        return float(np.mean(ranks)) if ranks else 0.0

    def reconstruction_map_strict_sampled(model, edges, n_negatives=500, seed=1234):
        rng = np.random.default_rng(seed)
        reciprocal_ranks = []
        for (u, v) in edges:
            if (u not in model.kv) or (v not in model.kv):
                continue
            neighbors = u_to_all_neighbors[u]
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < n_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate not in (u, v) and (candidate not in neighbors):
                    neg_candidates.append(candidate)
                attempts += 1
            candidates = neg_candidates + [v]
            dists = [(c, model.kv.distance(u, c)) for c in candidates]
            sorted_nodes = [x[0] for x in sorted(dists, key=lambda x: x[1])]
            try:
                r = sorted_nodes.index(v) + 1
                reciprocal_ranks.append(1.0 / r)
            except ValueError:
                pass
        return float(np.mean(reciprocal_ranks)) if reciprocal_ranks else 0.0

    def link_prediction_mean_rank(model, edges, num_negatives=50):
        rng = np.random.default_rng(1234)
        ranks = []
        for (source, target) in edges:
            if source not in vocab_nodes or target not in vocab_nodes:
                continue
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < num_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                
                if candidate != target and (source, candidate) not in all_edges_set:
                    neg_candidates.append(candidate)
                attempts += 1
            if not neg_candidates:
                continue
            candidates = neg_candidates + [target]
            dists = [model.kv.distance(source, c) for c in candidates]
            sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
            rank = sorted_candidates.index(target) + 1
            ranks.append(rank)
        return float(np.mean(ranks)) if ranks else 0.0

    def link_prediction_map(model, edges, num_negatives=50):
        rng = np.random.default_rng(1234)
        reciprocal_ranks = []
        for (source, target) in edges:
            if source not in vocab_nodes or target not in vocab_nodes:
                continue
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < num_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate != target and (source, candidate) not in all_edges_set:
                    neg_candidates.append(candidate)
                attempts += 1
            if not neg_candidates:
                continue
            candidates = neg_candidates + [target]
            dists = [model.kv.distance(source, c) for c in candidates]
            sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
            rank = sorted_candidates.index(target) + 1
            reciprocal_ranks.append(1.0 / rank)
        return float(np.mean(reciprocal_ranks)) if reciprocal_ranks else 0.0

    def precision_at_k(model, edges, k=10, num_negatives=50):
        rng = np.random.default_rng(1234)
        hits = 0
        count = 0
        for (source, target) in edges:
            if source not in vocab_nodes or target not in vocab_nodes:
                continue
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < num_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate != target and (source, candidate) not in all_edges_set:
                    neg_candidates.append(candidate)
                attempts += 1
            if not neg_candidates:
                continue
            candidates = neg_candidates + [target]
            dists = [model.kv.distance(source, c) for c in candidates]
            sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
            top_k_nodes = sorted_candidates[:k]
            if target in top_k_nodes:
                hits += 1
            count += 1
        return hits / count if count else 0.0

    def recall_at_k(model, edges, k=10, num_negatives=50):
        
        return precision_at_k(model, edges, k, num_negatives)

    
    recon_mr_strict = reconstruction_mean_rank_strict_sampled(
        model, train_relations, n_negatives=n_negatives_strict, seed=42
    )
    recon_map_strict = reconstruction_map_strict_sampled(
        model, train_relations, n_negatives=n_negatives_strict, seed=42
    )

    
    lp_mr = link_prediction_mean_rank(model, test_relations, num_negatives=50)
    lp_map_ = link_prediction_map(model, test_relations, num_negatives=50)
    lp_p10 = precision_at_k(model, test_relations, k=10, num_negatives=50)
    lp_r10 = recall_at_k(model, test_relations, k=10, num_negatives=50)

    
    return {
        "reconstruction_mean_rank_strict": recon_mr_strict,
        "reconstruction_map_strict": recon_map_strict,
        "lp_mean_rank": lp_mr,
        "lp_map": lp_map_,
        "lp_precision_10": lp_p10,
        "lp_recall_10": lp_r10,
    }



def run_multiple_experiments(
    address,                
    dim_list,               
    flip_list,              
    subset_size=10000,
    test_size=0.2,
    epochs=300,
    n_negatives_strict=500,
    seed=42
):
    
    train_relations_clean, test_relations, combined_relations = prepare_mammal_data(
        subset_size=subset_size,
        test_size=test_size,
        seed=seed
    )

    
    all_results = []

    for embedding_dim in dim_list:
        for flip_probability in flip_list:
            print("========================================================")
            print(f"RUNNING: dim={embedding_dim}, flip={flip_probability:.2f}")
            print("========================================================")

            
            flipped_train = flip_edge_directions(
                train_relations_clean,
                flip_probability=flip_probability,
                seed=seed
            )

            
            run_results = train_and_evaluate_poincare(
                train_relations=flipped_train,
                test_relations=test_relations,
                combined_relations=combined_relations,
                embedding_dim=embedding_dim,
                epochs=epochs,
                n_negatives_strict=n_negatives_strict
            )

            
            row_info = {
                "embedding_dim": embedding_dim,
                "flip_probability": flip_probability,
                "subset_size": subset_size,
                "test_size": test_size,
                "epochs": epochs,
                "n_negatives_strict": n_negatives_strict,
            }
            row_info.update(run_results)  
            all_results.append(row_info)

    
    df = pd.DataFrame(all_results)

    
    df.to_excel(address, index=False)
    print(f"\nAll experiment results saved to: {address}")
    return df



if __name__ == "__main__":
    dimension_list = [5, 10,20,50,100,200]          
    flip_probability_list = [0.0, 0.1, 0.3]  
    excel_path = "Y:\\Data Science Readings\\Geometry of Information\\MammalsExperiments\\mammal_exp.xlsx"
    final_df = run_multiple_experiments(
        address=excel_path,
        dim_list=dimension_list,
        flip_list=flip_probability_list,
        subset_size=1000000,        
        test_size=0.2,
        epochs=100,               
        n_negatives_strict=10,
        seed=42
    )
    print("\nFinal DataFrame of all runs:\n")
    print(final_df)


Building transitive closure for the Mammal subtree...
Total transitive hypernym pairs (mammal subtree): 7051
Train relations: 5640
Test relations : 1411
RUNNING: dim=5, flip=0.00
RUNNING: dim=5, flip=0.10
RUNNING: dim=5, flip=0.30
RUNNING: dim=10, flip=0.00
RUNNING: dim=10, flip=0.10
RUNNING: dim=10, flip=0.30
RUNNING: dim=20, flip=0.00
RUNNING: dim=20, flip=0.10
RUNNING: dim=20, flip=0.30
RUNNING: dim=50, flip=0.00
RUNNING: dim=50, flip=0.10
RUNNING: dim=50, flip=0.30
RUNNING: dim=100, flip=0.00
RUNNING: dim=100, flip=0.10
RUNNING: dim=100, flip=0.30
RUNNING: dim=200, flip=0.00
RUNNING: dim=200, flip=0.10
RUNNING: dim=200, flip=0.30

All experiment results saved to: Y:\Data Science Readings\Geometry of Information\MammalsExperiments\mammal_exp.xlsx

Final DataFrame of all runs:

    embedding_dim  flip_probability  subset_size  test_size  epochs  \
0               5               0.0      1000000        0.2     100   
1               5               0.1      1000000        0.2     100